# Chapter 2: 문서 로더와 벡터 저장소 실습

이 노트북은 LangChain의 문서 로더, 텍스트 분할기, 임베딩, 벡터 저장소를 실습합니다.

## 환경 설정

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. 텍스트 파일 로더

In [3]:
from langchain_community.document_loaders import TextLoader

# 샘플 텍스트 파일 생성
sample_text = """LangChain은 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.
다양한 컴포넌트를 제공하여 복잡한 AI 애플리케이션을 쉽게 만들 수 있습니다.
문서 로더, 텍스트 분할기, 임베딩, 벡터 저장소 등의 기능을 제공합니다."""

with open("sample.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# TextLoader 사용
loader = TextLoader("sample.txt", encoding="utf-8")
documents = loader.load()

print(f"로드된 문서 수: {len(documents)}")
print(f"문서 내용:\n{documents[0].page_content}")
print(f"메타데이터: {documents[0].metadata}")

로드된 문서 수: 1
문서 내용:
LangChain은 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.
다양한 컴포넌트를 제공하여 복잡한 AI 애플리케이션을 쉽게 만들 수 있습니다.
문서 로더, 텍스트 분할기, 임베딩, 벡터 저장소 등의 기능을 제공합니다.
메타데이터: {'source': 'sample.txt'}


## 2. 웹 페이지 로더

In [4]:
from langchain_community.document_loaders import WebBaseLoader

# 웹 페이지 로드
web_loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
web_docs = web_loader.load()

print(f"로드된 웹 문서 수: {len(web_docs)}")
print(f"첫 500자:\n{web_docs[0].page_content[:500]}...")
print(f"\n메타데이터: {web_docs[0].metadata}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


로드된 웹 문서 수: 1
첫 500자:





Introduction | 🦜️🔗 LangChain








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) A...

메타데이터: {'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}


## 3. PDF 로더

In [5]:
# PDF 파일이 있는 경우 실행
try:
    from langchain_community.document_loaders import PyPDFLoader
    
    # test.pdf 파일이 있다면 로드
    if os.path.exists("test.pdf"):
        pdf_loader = PyPDFLoader("test.pdf")
        pdf_docs = pdf_loader.load()
        print(f"PDF 페이지 수: {len(pdf_docs)}")
        print(f"첫 페이지 내용 (100자):\n{pdf_docs[0].page_content[:100]}...")
    else:
        print("test.pdf 파일이 없습니다.")
except ImportError:
    print("PyPDF 라이브러리가 설치되지 않았습니다. 'pip install pypdf'로 설치하세요.")

PDF 페이지 수: 1
첫 페이지 내용 (100자):
Life in ancient Greece was centered around the polis, or city-state, which served as the heart of 
s...


## 4. 재귀적 텍스트 분할기

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 긴 텍스트 생성
long_text = """
LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.

주요 기능:
1. 프롬프트 관리: 프롬프트 템플릿을 쉽게 만들고 관리할 수 있습니다.
2. 체인: 여러 컴포넌트를 연결하여 복잡한 워크플로우를 구성할 수 있습니다.
3. 에이전트: LLM이 도구를 사용하여 작업을 수행할 수 있게 합니다.
4. 메모리: 대화 컨텍스트를 유지하고 관리합니다.

LangChain의 장점:
- 모듈성: 각 컴포넌트를 독립적으로 사용하거나 조합할 수 있습니다.
- 확장성: 커스텀 컴포넌트를 쉽게 추가할 수 있습니다.
- 통합: 다양한 LLM 제공자와 도구를 지원합니다.
"""

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)

print(f"분할된 청크 수: {len(chunks)}\n")
for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i} ({len(chunk)}자):\n{chunk}\n{'-'*50}")

분할된 청크 수: 5

청크 1 (54자):
LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.
--------------------------------------------------
청크 2 (91자):
주요 기능:
1. 프롬프트 관리: 프롬프트 템플릿을 쉽게 만들고 관리할 수 있습니다.
2. 체인: 여러 컴포넌트를 연결하여 복잡한 워크플로우를 구성할 수 있습니다.
--------------------------------------------------
청크 3 (69자):
3. 에이전트: LLM이 도구를 사용하여 작업을 수행할 수 있게 합니다.
4. 메모리: 대화 컨텍스트를 유지하고 관리합니다.
--------------------------------------------------
청크 4 (85자):
LangChain의 장점:
- 모듈성: 각 컴포넌트를 독립적으로 사용하거나 조합할 수 있습니다.
- 확장성: 커스텀 컴포넌트를 쉽게 추가할 수 있습니다.
--------------------------------------------------
청크 5 (29자):
- 통합: 다양한 LLM 제공자와 도구를 지원합니다.
--------------------------------------------------


## 5. 코드 분할기

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

# 파이썬 코드 예제
python_code = '''
def hello_world():
    """간단한 헬로 월드 함수"""
    print("Hello, World!")
    return True

class Greeting:
    """인사말 클래스"""
    def __init__(self, name):
        self.name = name
    
    def say_hello(self):
        """인사하기"""
        return f"Hello, {self.name}!"
    
    def say_goodbye(self):
        """작별 인사"""
        return f"Goodbye, {self.name}!"

if __name__ == "__main__":
    hello_world()
    greeter = Greeting("LangChain")
    print(greeter.say_hello())
'''

# 파이썬 코드 분할기
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=150,
    chunk_overlap=20
)

code_chunks = python_splitter.split_text(python_code)

print(f"코드 청크 수: {len(code_chunks)}\n")
for i, chunk in enumerate(code_chunks, 1):
    print(f"코드 청크 {i}:\n{chunk}\n{'-'*50}")

코드 청크 수: 5

코드 청크 1:
def hello_world():
    """간단한 헬로 월드 함수"""
    print("Hello, World!")
    return True
--------------------------------------------------
코드 청크 2:
class Greeting:
    """인사말 클래스"""
    def __init__(self, name):
        self.name = name
--------------------------------------------------
코드 청크 3:
def say_hello(self):
        """인사하기"""
        return f"Hello, {self.name}!"
--------------------------------------------------
코드 청크 4:
def say_goodbye(self):
        """작별 인사"""
        return f"Goodbye, {self.name}!"
--------------------------------------------------
코드 청크 5:
if __name__ == "__main__":
    hello_world()
    greeter = Greeting("LangChain")
    print(greeter.say_hello())
--------------------------------------------------


## 6. 마크다운 분할기

In [8]:
from langchain.text_splitter import MarkdownTextSplitter

markdown_text = """
# LangChain 가이드

## 소개
LangChain은 LLM 애플리케이션 개발 프레임워크입니다.

## 주요 기능

### 프롬프트 템플릿
동적 프롬프트 생성을 지원합니다.

### 체인
여러 컴포넌트를 연결합니다.

## 시작하기
```python
from langchain import OpenAI
llm = OpenAI()
```
"""

# 마크다운 분할기
md_splitter = MarkdownTextSplitter(chunk_size=100, chunk_overlap=10)
md_chunks = md_splitter.split_text(markdown_text)

print(f"마크다운 청크 수: {len(md_chunks)}\n")
for i, chunk in enumerate(md_chunks, 1):
    print(f"MD 청크 {i}:\n{chunk}\n{'-'*50}")

마크다운 청크 수: 4

MD 청크 1:
# LangChain 가이드

## 소개
LangChain은 LLM 애플리케이션 개발 프레임워크입니다.

## 주요 기능
--------------------------------------------------
MD 청크 2:
## 주요 기능

### 프롬프트 템플릿
동적 프롬프트 생성을 지원합니다.

### 체인
여러 컴포넌트를 연결합니다.
--------------------------------------------------
MD 청크 3:
## 시작하기
```python
from langchain import OpenAI
llm = OpenAI()
--------------------------------------------------
MD 청크 4:
```
--------------------------------------------------


## 7. 임베딩

In [9]:
from langchain_openai import OpenAIEmbeddings
import numpy as np

# 임베딩 모델 생성
embeddings = OpenAIEmbeddings()

# 텍스트 임베딩
texts = [
    "LangChain은 AI 애플리케이션 프레임워크입니다.",
    "파이썬은 프로그래밍 언어입니다.",
    "LangChain을 사용하면 LLM 애플리케이션을 쉽게 만들 수 있습니다."
]

# 임베딩 생성
embedded_texts = embeddings.embed_documents(texts)

print(f"임베딩 차원: {len(embedded_texts[0])}")
print(f"첫 번째 텍스트의 임베딩 (처음 10개 값): {embedded_texts[0][:10]}")

# 유사도 계산 (코사인 유사도)
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("\n텍스트 간 유사도:")
for i in range(len(texts)):
    for j in range(i+1, len(texts)):
        sim = cosine_similarity(embedded_texts[i], embedded_texts[j])
        print(f"텍스트 {i+1} <-> 텍스트 {j+1}: {sim:.4f}")

임베딩 차원: 1536
첫 번째 텍스트의 임베딩 (처음 10개 값): [-0.006174771580845118, -0.024400196969509125, -0.019658733159303665, -0.03282342851161957, 0.018368078395724297, 0.00598796596750617, -0.01618075557053089, 0.02467191405594349, 0.008402852341532707, 0.010433937422931194]

텍스트 간 유사도:
텍스트 1 <-> 텍스트 2: 0.8110
텍스트 1 <-> 텍스트 3: 0.9208
텍스트 2 <-> 텍스트 3: 0.7723


## 8. 문서 로드, 분할, 임베딩 통합

In [10]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 생성
sample_content = """
인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 과학의 한 분야입니다.

머신러닝은 AI의 한 분야로, 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구합니다.

딥러닝은 머신러닝의 한 방법으로, 인공신경망을 여러 층으로 쌓아 복잡한 패턴을 학습합니다.

자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 처리할 수 있도록 하는 AI 기술입니다.
"""

with open("ai_intro.txt", "w", encoding="utf-8") as f:
    f.write(sample_content)

# 1. 문서 로드
loader = TextLoader("ai_intro.txt", encoding="utf-8")
documents = loader.load()

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)
splits = text_splitter.split_documents(documents)

# 3. 임베딩 생성
embeddings = OpenAIEmbeddings()
embedded_docs = embeddings.embed_documents([doc.page_content for doc in splits])

print(f"원본 문서 길이: {len(documents[0].page_content)}자")
print(f"분할된 청크 수: {len(splits)}")
print(f"각 청크의 임베딩 차원: {len(embedded_docs[0])}\n")

for i, split in enumerate(splits):
    print(f"청크 {i+1}: {split.page_content[:50]}...")

원본 문서 길이: 226자
분할된 청크 수: 4
각 청크의 임베딩 차원: 1536

청크 1: 인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현한 컴퓨터 과학의 ...
청크 2: 머신러닝은 AI의 한 분야로, 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘...
청크 3: 딥러닝은 머신러닝의 한 방법으로, 인공신경망을 여러 층으로 쌓아 복잡한 패턴을 학습합니다....
청크 4: 자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 처리할 수 있도록 하는 AI 기술입니...


## 9. FAISS 벡터 저장소

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서들
texts = [
    "파이썬은 배우기 쉬운 프로그래밍 언어입니다.",
    "LangChain은 LLM 애플리케이션 개발 프레임워크입니다.",
    "벡터 데이터베이스는 임베딩을 저장하고 검색합니다.",
    "파이썬으로 AI 애플리케이션을 개발할 수 있습니다.",
    "FAISS는 Facebook이 개발한 벡터 검색 라이브러리입니다."
]

# 벡터 저장소 생성
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embeddings)

# 유사도 검색
query = "파이썬 프로그래밍"
results = vectorstore.similarity_search(query, k=3)

print(f"쿼리: '{query}'\n")
print("유사한 문서 (상위 3개):")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

# 점수와 함께 검색
print("\n점수와 함께 검색:")
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)
for doc, score in results_with_scores:
    print(f"점수: {score:.4f} - {doc.page_content}")

쿼리: '파이썬 프로그래밍'

유사한 문서 (상위 3개):
1. 파이썬은 배우기 쉬운 프로그래밍 언어입니다.
2. 파이썬으로 AI 애플리케이션을 개발할 수 있습니다.
3. LangChain은 LLM 애플리케이션 개발 프레임워크입니다.

점수와 함께 검색:
점수: 0.1417 - 파이썬은 배우기 쉬운 프로그래밍 언어입니다.
점수: 0.1853 - 파이썬으로 AI 애플리케이션을 개발할 수 있습니다.
점수: 0.4124 - LangChain은 LLM 애플리케이션 개발 프레임워크입니다.


## 10. 벡터 저장소 저장 및 로드

In [12]:
# 벡터 저장소 저장
vectorstore.save_local("faiss_index")
print("벡터 저장소가 'faiss_index' 디렉토리에 저장되었습니다.")

# 벡터 저장소 로드
loaded_vectorstore = FAISS.load_local(
    "faiss_index", 
    embeddings,
    allow_dangerous_deserialization=True
)
print("벡터 저장소를 로드했습니다.")

# 로드된 저장소로 검색
query = "벡터 검색"
results = loaded_vectorstore.similarity_search(query, k=2)
print(f"\n쿼리: '{query}'")
print("검색 결과:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

벡터 저장소가 'faiss_index' 디렉토리에 저장되었습니다.
벡터 저장소를 로드했습니다.

쿼리: '벡터 검색'
검색 결과:
1. 벡터 데이터베이스는 임베딩을 저장하고 검색합니다.
2. FAISS는 Facebook이 개발한 벡터 검색 라이브러리입니다.


## 11. 메타데이터 필터링

In [13]:
from langchain.schema import Document

# 메타데이터가 있는 문서들
docs = [
    Document(
        page_content="파이썬은 다양한 분야에서 사용됩니다.",
        metadata={"category": "programming", "language": "python"}
    ),
    Document(
        page_content="자바스크립트는 웹 개발에 필수적입니다.",
        metadata={"category": "programming", "language": "javascript"}
    ),
    Document(
        page_content="머신러닝은 데이터에서 패턴을 학습합니다.",
        metadata={"category": "ai", "topic": "ml"}
    ),
    Document(
        page_content="딥러닝은 신경망을 사용합니다.",
        metadata={"category": "ai", "topic": "dl"}
    ),
]

# 메타데이터와 함께 벡터 저장소 생성
vectorstore_with_metadata = FAISS.from_documents(docs, embeddings)

# 메타데이터 필터링 검색
query = "프로그래밍 언어"
filter_dict = {"category": "programming"}

# 필터 적용 검색 (FAISS는 직접적인 필터링을 지원하지 않으므로 전체 검색 후 필터링)
all_results = vectorstore_with_metadata.similarity_search(query, k=10)
filtered_results = [
    doc for doc in all_results 
    if doc.metadata.get("category") == "programming"
]

print(f"쿼리: '{query}'")
print(f"필터: category='programming'\n")
print("필터링된 결과:")
for doc in filtered_results:
    print(f"- {doc.page_content}")
    print(f"  메타데이터: {doc.metadata}")

쿼리: '프로그래밍 언어'
필터: category='programming'

필터링된 결과:
- 파이썬은 다양한 분야에서 사용됩니다.
  메타데이터: {'category': 'programming', 'language': 'python'}
- 자바스크립트는 웹 개발에 필수적입니다.
  메타데이터: {'category': 'programming', 'language': 'javascript'}


## 실습 과제

다음 과제들을 시도해보세요:

1. 여러 파일을 로드하고 통합하여 벡터 저장소 만들기
2. 다양한 청크 크기로 실험하고 검색 품질 비교
3. 커스텀 메타데이터를 추가하여 고급 필터링 구현

In [ ]:
# 여기에 실습 코드를 작성하세요
